In [ ]:
!pip install langchain --quiet
!pip install google-ai-generativelanguage --quiet


In [ ]:
!pip install google-ai-generativelanguage --quiet


In [ ]:
!pip install langchain_google_genai --quiet

In [ ]:
! pip install langchain-community --quiet

In [ ]:
import os
#google API key
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_flash = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    temperature=0.2,
    top_p=1
)


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#loading synthetic text hospital data
loader = TextLoader("/content/Synthetic-data-RAG.txt")
docs = loader.load()

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"Number of chunks: {len(chunks)}")
print(f"Sample chunk:\n{chunks[0].page_content}")


Number of chunks: 139
Sample chunk:
1. General Hospital Information

Department Details:

Department: Neurology

Head of Department: Dr. Eleanor Vance, MD, PhD

Location: North Wing, 3rd Floor

Contact Phone: (555) 123-4567


In [ ]:
! pip install faiss-cpu --quiet

In [ ]:

for i in range(min(5, len(chunks))):
    print(f"Sample chunk {i+1}:\n{chunks[i].page_content}\n---")

Sample chunk 1:
1. General Hospital Information

Department Details:

Department: Neurology

Head of Department: Dr. Eleanor Vance, MD, PhD

Location: North Wing, 3rd Floor

Contact Phone: (555) 123-4567
---
Sample chunk 2:
Contact Phone: (555) 123-4567

Services: Diagnosis and treatment of disorders of the nervous system, including brain, spinal cord, and peripheral nerves. Specializes in stroke, epilepsy, multiple sclerosis, Parkinson's disease, and neuro-oncology.

Sub-specialties: Neuro-oncology Clinic, Stroke Unit, Epilepsy Monitoring Unit, Movement Disorders Clinic.

Department: Oncology
---
Sample chunk 3:
Department: Oncology

Head of Department: Dr. Ben Carter, MD

Location: East Wing, 2nd Floor

Contact Phone: (555) 987-6543

Services: Comprehensive cancer care including chemotherapy, immunotherapy, targeted therapy, and palliative care. Works in conjunction with Radiation Oncology and Surgical Oncology.
---
Sample chunk 4:
Sub-specialties: Medical Oncology, Hematologic Malig

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embedding model (Gemini)
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# vector store from the document chunks
vectorstore = FAISS.from_documents(chunks, embedding_model)

vectorstore.save_local("HospitalDB")


In [ ]:
retrieved_docs = vectorstore.similarity_search("What are the common symptoms of anxiety?")

print("Retrieved documents based on a sample query:")
for doc in retrieved_docs:
    print(f"Content:\n{doc.page_content}\nMetadata: {doc.metadata}\n---")

Retrieved documents based on a sample query:
Content:
Fatigue: This is very common. Rest when you need to, but try to incorporate gentle exercise like short walks. Prioritize tasks and ask for help.

Hair Loss: If your chemo causes hair loss, it usually starts 2-3 weeks after treatment begins. Consider wigs, scarves, or hats. Hair usually regrows after treatment ends.
Metadata: {'source': '/content/Synthetic-data-RAG.txt'}
---
Content:
Common Symptoms: Symptoms depend on the tumor's location and size but can include headaches, seizures, weakness or numbness in limbs, speech difficulties, vision changes, and personality changes.

Diagnosis: Diagnosis typically involves a neurological exam, MRI or CT scans, and a biopsy (surgical removal of a tissue sample for examination).
Metadata: {'source': '/content/Synthetic-data-RAG.txt'}
---
Content:
Q: Will I lose my hair?

A: Hair loss (alopecia) is common but usually temporary and typically occurs only in the area being treated. Your radiation

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Question
question = "who is Head of the department of Neurology?"
retriever = vectorstore.as_retriever()


docs = retriever.get_relevant_documents(question)
context = "\n\n".join(doc.page_content for doc in docs)

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using ONLY the provided context.\n\nContext:\n{context}"),
    ("human", "{question}")
])

chain = rag_prompt | gemini_flash

response = chain.invoke({"context": context, "question": question})
print(response.content)


<ipython-input-13-124b8b5a61e1>:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


Dr. Eleanor Vance, MD, PhD
